# Laboratorium 1 - analiza koszykowa

## Przygotowanie

 * pobierz i wypakuj dataset: https://kaggle.com/datasets/rashikrahmanpritom/groceries-dataset-for-market-basket-analysismba?resource=download&select=basket.csv
   * alternatywnie, pobierz plik `basket.csv` z Teamsów
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install more-itertools`

## Część 1. - przygotowanie danych

In [4]:
from email.policy import default

# importujemy wszystkie potrzebne pakiety

from more_itertools import powerset

In [5]:
# definiujemy stale

PATH = './basket.csv'
EPSILON = 0.001
K = 4

In [6]:
# wczytujemy dane o koszykach

def read_baskets(path: str) -> list[tuple[str]]:
    with open(path) as f:
        raw = f.read()
    baskets = [set([y.lower() for y in x.split(',') if y]) for x in raw.split('\n')[1:] if x]
    return baskets

def unique_products(baskets: list[tuple[str]]) -> list[str]:
    products = set()
    for basket in baskets:
        products.update(basket)
    return sorted(list(products))

baskets = read_baskets(PATH)
products = unique_products(baskets)

## Część 2. - obliczanie wskaźników

In [22]:
from collections import defaultdict


# obliczamy strukture danych (np. slownik albo graf) przechowujaca wszystkie interesujace wartosci `support`

def get_supports(baskets: list[tuple[str]], all_products: list[str], epsilon: float) -> dict[frozenset[str], float]:
    supports = {}
    baskets_by_products = _get_baskets_by_product(baskets)
    
    for basket in baskets:
        for sub_basket in powerset(basket):
            if len(sub_basket) > K:
                break
                
            sub_basket_set = frozenset(sub_basket)
            if sub_basket_set in supports:
                continue
                
            occurrences = 0
            for product in sub_basket:
                for b in baskets_by_products[product]:
                    if sub_basket_set.issubset(set(b)):
                        # basket will occur for every product in sub_basket
                        # so we need to divide `occurrences` by the number of products in sub_basket
                        occurrences += 1
            
            support = occurrences / len(baskets)
            if len(sub_basket) > 0:
                support /= len(sub_basket)
                
            # ideally if support of sub_basket is less than epsilon, we should
            # not calculate the values for baskets that contain this sub_basket
            if support <= epsilon: 
                continue
                
            supports[sub_basket_set] = support

            
    return supports
            
            
def _get_baskets_by_product(baskets: list[tuple[str]]) -> dict[str, list[tuple[str]]]:
    baskets_by_products = defaultdict(list)
    for basket in baskets:
        for product in basket:
            baskets_by_products[product].append(basket)
            
    return baskets_by_products
                
                
    
supports = get_supports(baskets, products, EPSILON)
supports

{frozenset({'pastry'}): 0.0517275947336764,
 frozenset({'whole milk'}): 0.15792287642852368,
 frozenset({'salty snack'}): 0.018779656485998796,
 frozenset({'pastry', 'whole milk'}): 0.006482657221145492,
 frozenset({'salty snack', 'whole milk'}): 0.0019381140145692708,
 frozenset({'semi-finished bread'}): 0.009490075519615051,
 frozenset({'yogurt'}): 0.08587850030074183,
 frozenset({'sausage'}): 0.06034886052262247,
 frozenset({'semi-finished bread', 'whole milk'}): 0.001670787943594199,
 frozenset({'whole milk', 'yogurt'}): 0.011160863463209249,
 frozenset({'sausage', 'whole milk'}): 0.008955423377664907,
 frozenset({'sausage', 'yogurt'}): 0.005747510525964045,
 frozenset({'sausage', 'whole milk', 'yogurt'}): 0.0014702933903628951,
 frozenset({'pickled vegetables'}): 0.008955423377664907,
 frozenset({'soda'}): 0.09710619528169484,
 frozenset({'canned beer'}): 0.04691572545612511,
 frozenset({'misc. beverages'}): 0.01577223818752924,
 frozenset({'hygiene articles'}): 0.0137004611374724

In [23]:
len(supports)

750

In [24]:
# definiujemy funkcje obliczajace support, confidence i lift

def support(supports: dict[frozenset[str], float], products: tuple[str]) -> float:
    return supports.get(frozenset(products), 0.0)


def confidence(supports: dict[frozenset[str], float], prior_products: tuple[str], following_products: tuple[str]) -> float:
    try:
        return support(supports, prior_products + following_products) / support(supports, prior_products)
    except ZeroDivisionError:
        return 0.0


def lift(supports:  dict[frozenset[str], float], prior_products: tuple[str], following_products: tuple[str]) -> float:
    try:
        return support(supports, prior_products + following_products) / (support(supports, prior_products) * support(supports, following_products))
    except ZeroDivisionError:
        return 0.0

In [25]:
print(support(supports, ('whole milk', 'rolls/buns')))
print(confidence(supports, ('whole milk', 'rolls/buns'), ('yogurt',)))
print(lift(supports, ('whole milk', 'rolls/buns'), ('yogurt',)))

0.013967787208447505
0.09569377990430622
1.1142926293448512


## Część 3. - generowanie rekomendacji

In [26]:
# wyznaczamy liste potencjalnych rekomendacji
# rekomendowane artykuly powinny miec lift > 1 i jak najwyzszy confidence

def generate_basic_candidates(basket: tuple[str], products: list[str], supports: dict[frozenset[str], float]) -> list[tuple[str, tuple[str], float, float]]:
    candidates: list[tuple[str, tuple[str], float, float]] = []
    for sub_basket in powerset(basket):
        if len(sub_basket) == 0:
            continue
        for product in products:
            if product in basket:
                continue
            confidence_val = confidence(supports, sub_basket, sub_basket + (product,))
            lift_val = lift(supports, sub_basket, sub_basket + (product,))

            if lift_val > 1:
                candidates.append((product, sub_basket, confidence_val, lift_val))

    return sorted(candidates, key=lambda x: x[2], reverse=True)

In [27]:
# zaproponuj drugi, bardziej zaawansowany algorytm, np.:
# - jesli produkt X wystepuje w liscie kandydatow kilkukrotnie, oblicz srednia lub iloczyn confidence
# - posortuj kandydatow po iloczynie configence i lift

def generate_advanced_candidates(basket: tuple[str], products: list[str], supports) -> list[tuple[str, tuple[str] | None, float, float]]:
    candidates = generate_basic_candidates(basket, products, supports)
    product_confidences = defaultdict(list)
    
    for product, _, confidence, _ in candidates:
        product_confidences[product].append(confidence)
        
    for product, confidences in product_confidences.items():
        candidates.append(
            (product, None, sum(confidences) / len(confidences), 1)
        )
    
    return sorted(candidates, key=lambda x: x[2] * x[3], reverse=True)


In [28]:
print(baskets[1])
generate_basic_candidates(tuple(baskets[1]), products, supports)
generate_advanced_candidates(tuple(baskets[1]), products, supports)

{'semi-finished bread', 'whole milk', 'yogurt', 'sausage'}


[('rolls/buns',
  ('whole milk', 'sausage'),
  0.12686567164179105,
  111.6641791044776),
 ('soda', ('whole milk', 'sausage'), 0.11940298507462685, 111.66417910447761),
 ('other vegetables',
  ('semi-finished bread',),
  0.1056338028169014,
  105.37323943661971),
 ('rolls/buns',
  ('whole milk', 'yogurt'),
  0.11976047904191618,
  89.59880239520959),
 ('other vegetables',
  ('whole milk', 'yogurt'),
  0.10179640718562874,
  89.59880239520959),
 ('other vegetables', ('sausage',), 0.09966777408637872, 16.570321151716502),
 ('soda', ('sausage',), 0.09856035437430785, 16.570321151716502),
 ('rolls/buns', ('sausage',), 0.08859357696566998, 16.570321151716502),
 ('other vegetables', ('yogurt',), 0.09416342412451362, 11.644357976653696),
 ('rolls/buns', ('yogurt',), 0.09105058365758756, 11.644357976653696),
 ('bottled beer', ('sausage',), 0.05537098560354374, 16.5703211517165),
 ('root vegetables', ('sausage',), 0.05537098560354374, 16.5703211517165),
 ('pastry', ('sausage',), 0.0531561461794

In [29]:
print(baskets[33])
generate_basic_candidates(tuple(baskets[33]), products, supports)
generate_advanced_candidates(tuple(baskets[33]), products, supports)

{'root vegetables', 'domestic eggs', 'soda', 'white wine', 'photo/film', 'tropical fruit', 'yogurt'}


[('whole milk', ('white wine',), 0.10857142857142857, 85.50285714285714),
 ('whole milk', ('domestic eggs',), 0.14234234234234236, 26.96036036036036),
 ('other vegetables',
  ('domestic eggs',),
  0.0954954954954955,
  26.96036036036036),
 ('rolls/buns', ('domestic eggs',), 0.0918918918918919, 26.96036036036036),
 ('whole milk', ('tropical fruit',), 0.121301775147929, 14.756410256410257),
 ('whole milk', ('root vegetables',), 0.10854947166186359, 14.373679154658982),
 ('bottled water',
  ('domestic eggs',),
  0.057657657657657665,
  26.96036036036036),
 ('whole milk', ('yogurt',), 0.1299610894941634, 11.644357976653696),
 ('other vegetables',
  ('tropical fruit',),
  0.09270216962524655,
  14.756410256410257),
 ('rolls/buns', ('tropical fruit',), 0.08974358974358976, 14.756410256410257),
 ('sausage', ('domestic eggs',), 0.04864864864864865, 26.960360360360365),
 ('whole milk', ('soda',), 0.11975223675154853, 10.298004129387476),
 ('bottled beer', ('domestic eggs',), 0.04504504504504505